# Code to generate Figure 1 from the paper

In [ ]:
# Copyright (C) to Yingcong Tan, Andrew Delong, Daria Terekhov. All Rights Reserved.


import numpy as np
import torch
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.max_open_warning'] = 0
%matplotlib inline
import deep_inv_opt as io
import deep_inv_opt.plot as iop

as_tensor = io.as_tensor
as_numpy = io.as_numpy
as_str = io.as_str

# to save the figure images to disk, override this with something like "~/Downloads"  
FIGURE_SAVE_DIR = "C:/Users/Public/Downloads/"  

# Figure 1A "learn c only"
This sub-figure depicts a classical inverse optimization problem: infer the *c* vector from a single observation `x_train`. There are closed form solutions to this in terms of the constraint coefficients, but we can also learn a good c vector by gradient descent on a squared error loss. Even though we are using the ParametricLP machinery to encode the c-vector with a single coefficient `w0`, that is just to make the example tidier -- it is not really a parametric LP being inversely optimized.

In [ ]:
class Figure1A_PLP(io.ParametricLP):
    
    def generate(self, u, w):
        (w0,) = w
        
        c = [[torch.cos(w0)],
             [torch.sin(w0)]]

        A_ub = [[-1.0,  0.0],  # x0 >= 0
                [ 0.0, -1.0],  # x1 >= 0
                [ 1.0,  0.0],  # x0 <= 1
                [ 0.0,  1.0],  # x1 <= 1
                [ 1.0,  1.5]]  # x0 + x1 <= ...

        b_ub = [[ 0.0 ],
                [ 0.0 ],
                [ 1.0 ],
                [ 0.75 ],
                [ 1.5]]
        
        return c, A_ub, b_ub, None, None
    
    def __call__(self, u=None):
        if u is None:
            u = as_tensor([[0.0]])  # Dummy u, not used
        return super().__call__(u)
    
    def plot(self, color='k', fig=None, linestyle=None, want_constraints=True):
        with torch.no_grad():
            # Set up the plot
            if fig is None:
                fig = plt.figure(dpi=100, figsize=(5, 5))
                ax = fig.add_subplot(111)
                ax.set_xlim(-0.35, 1.25)
                ax.set_ylim(-0.35, 1.25)
                
                ax.set_aspect('equal', 'box')
                ax.set_axis_off()
                fig.tight_layout()
            else:
                ax = fig.gca()

            # Plot each set of constraints
            c, A_ub, b_ub, _, _ = as_numpy(*self())            
            iop.plot_linear_program(0.25*c,
                                    A_ub if want_constraints else None,
                                    b_ub if want_constraints else None,
                                    color=color, linestyle=linestyle, cxy=(0.85, 1.0))
            return fig, ax
            

##########################################
        
f_true  = Figure1A_PLP([np.pi*6/4+0.08])  # True c angle
f_learn = Figure1A_PLP([3.8])

u_train = io.tensor([[0.0]])
x_train = torch.cat([io.linprog(*f_true(), eps=0.0000001).detach().t() for ui in u_train])

# Plot c_true and constraints
fig, ax = f_true.plot(color='k')
iop.plot_targets(x_train, 'ok', markerfacecolor='white', markersize=14.0, markeredgewidth=1.5)

# Plot c_init
f_learn.plot(color=[0.1, 0.7, 0.0], fig=fig, want_constraints=False)
iop.plot_targets(torch.cat([io.linprog(*f_learn(), eps=0.0000001).detach().t() for ui in u_train]), 'o', color=[0.1,0.7,0.0], markerfacecolor='white', markersize=10.0, markeredgewidth=1.5)

lr = [10]
max_steps = 20
io.inverse_parametric_linprog(u_train, x_train, f_learn, lr=lr, max_steps=max_steps,
                              callback=io.inverse_parametric_linprog_step_printer(),
                              solver=io.custom_linprog(eps=0.001))

# Plot c_learned
f_learn.plot(color=[0, 0, 0.9], fig=fig, want_constraints=False)
iop.plot_targets(torch.cat([io.linprog(*f_learn(), eps=0.0000001).detach().t() for ui in u_train]), 'o', color=[0.0,0.0,0.9], markerfacecolor='white', markersize=6.0, markeredgewidth=1.5)

if FIGURE_SAVE_DIR:
    plt.savefig(FIGURE_SAVE_DIR + '/deepinverse_fig1_cvector.pdf', bbox_inches='tight', dpi=100)

# Figure 1B - "learn c, A, b jointly"

In [ ]:
class Figure1B_PLP(io.ParametricLP):
    
    def __init__(self, wc, wA, wb):
        # wc should be angle of c
        # wA should be coefficients for last three rows of A
        # wb should be coefficients for last three rows of b
        super().__init__(torch.cat([_.view(-1) for _ in as_tensor(wc, wA, wb)]))
    
    def generate(self, u, w):
        wc = w[0:1].view(1, 1)
        wA = w[1:7].view(3, 2)
        wb = w[7:10].view(3, 1)
        
        c = [[torch.cos(wc)],
             [torch.sin(wc)]]

        A_ub = [[-1.0,  0.0],  # x0 >= 0
                [ 0.0, -1.0],  # x1 >= 0
                [ wA[0,0],  wA[0,1]],  # x0 <= 1
                [ wA[1,0],  wA[1,1]],  # x1 <= 1
                [ wA[2,0],  wA[2,1]]]  # x0 + x1 <= ...

        b_ub = [[ 0.0 ],
                [ 0.0 ],
                [ wb[0,0] ],
                [ wb[1,0] ],
                [ wb[2,0]]]
        
        return c, A_ub, b_ub, None, None
    
    def __call__(self, u=None):
        if u is None:
            u = as_tensor([[0.0]])  # Dummy u, not used
        return super().__call__(u)
    
    def plot(self, color='k', fig=None, linestyle=None, want_constraints=True):
        with torch.no_grad():
            # Set up the plot
            if fig is None:
                fig = plt.figure(dpi=100, figsize=(5, 5))
                ax = fig.add_subplot(111)
                ax.set_xlim(-0.35, 1.25)
                ax.set_ylim(-0.35, 1.25)
                
                ax.set_aspect('equal', 'box')
                ax.set_axis_off()
                fig.tight_layout()
            else:
                ax = fig.gca()

            # Plot each set of constraints
            c, A_ub, b_ub, _, _ = as_numpy(*self())
            iop.plot_linear_program(0.25*c,
                                    A_ub if want_constraints else None,
                                    b_ub if want_constraints else None,
                                    color=color, linestyle=linestyle, cxy=(1.2, 1.0))
            return fig, ax


        
f_true  = Figure1B_PLP(
    wc=[np.pi*6/4-0.05],
    wA=[[1.0, 0.0],
        [0.0, 1.0],
        [1.3, 1.0]],
    wb=[[1.0],
        [1.0],
        [1.5]])

import copy

f_init  = Figure1B_PLP(
    wc=[3.5],
    wA=[[1.0, 0.0],
        [0.0, 1.0],
        [1.0, 1.5]],
    wb=[[1.0],
        [0.75],
        [1.5]])

u_train = io.tensor([[0.0]])
x_train = torch.cat([io.linprog(*f_true(), eps=0.0000001).detach().t() for ui in u_train])

# Custom learning rates to get the particular solution that looks interesting,
# (Since learning constraints from single point is so under-determined, we can get lots of different
#  solutions for constraints depending on initial c vector and these learning rates etc.)
lr = [200,    # wc
      .1, .1,  # wA
      .1, .1,
      .1, .1,
      .1,     # wB
      .1,
      .1]
max_steps = 50
f_learn = copy.deepcopy(f_init)  # In order to plot f_init and f_learn separately and in reverse order, make a copy here.

io.inverse_parametric_linprog(u_train, x_train, f_learn, lr=lr, max_steps=max_steps,
                              callback=io.inverse_parametric_linprog_step_printer(),
                              solver=io.custom_linprog(eps=0.001))

# Do the actual plotting, finishing with f_true (black constraints on top)
fig, ax = f_init.plot(color=[0.1, 0.7, 0.0])
f_learn.plot(color=[0, 0, 0.9], fig=fig)
f_true.plot(color='k', fig=fig)

# Plot the target points
iop.plot_targets(x_train, 'ok', markerfacecolor='white', markersize=14.0, markeredgewidth=1.5)
iop.plot_targets(torch.cat([io.linprog(*f_init(), eps=0.0000001).detach().t() for ui in u_train]), 'o', color=[0.1,0.7,0.0], markerfacecolor='white', markersize=10.0, markeredgewidth=1.5)
iop.plot_targets(torch.cat([io.linprog(*f_learn(), eps=0.0000001).detach().t() for ui in u_train]), 'o', color=[0.0,0.0,0.9], markerfacecolor='white', markersize=6.0, markeredgewidth=1.5)

if FIGURE_SAVE_DIR:
    plt.savefig(FIGURE_SAVE_DIR + '/deepinverse_fig1_constraints.pdf', bbox_inches='tight', dpi=100)

# Figure 1C - "parametric LP"

In [ ]:
class Figure1C_PLP(io.ParametricLP):
    
    def generate(self, u, w):
        w0, w1 = w

        c = [[torch.cos(w0 + w1*u)],
             [torch.sin(w0 + w1*u)]]

        A_ub = [[-1.0,  0.0],
                [ 0.0, -1.0],
                [ w0,  1.0 + 1/3*w1*u]]

        b_ub = [[ 0.0 + 0.2*w0*u],
                [ 0.0 - 0.2*w1*u],
                [ w0 + 0.1*u]]
        
        return c, A_ub, b_ub, None, None
    
    def plot(self, u, color='k', alpha=1.0, linestyle=None):
        # Set up the plot
        fig = plt.figure(dpi=100, figsize=(5, 5))
        ax = fig.add_subplot(111)
        ax.set_xlim(-0.35, 1.25)
        ax.set_ylim(-0.35, 1.25)
        ax.set_aspect('equal', 'box')
        ax.set_axis_off()
        fig.tight_layout()
        
        # Plot each set of constraints
        for i, ui in enumerate(u):
            # Alpha progresses from initial alpha to 1.0 as we go through the points
            alpha_i = alpha + (1-alpha)*(i/(len(u)-1) if len(u) > 1 else 1)
            
            # Find the LP for this particular u
            c, A_ub, b_ub, _, _ = as_numpy(*self(ui))
            iop.plot_linear_program(0.25*c, A_ub, b_ub, color=color, alpha=alpha_i, linestyle=linestyle, cxy=(0.95, 0.95))

        return fig, ax

    
###########################    
        
# These are the values of u that will be plotted and trained on
u = io.tensor([[-1.5],
               [-0.5],
               [ 0.5],
               [ 1.5]])
        
f_true = Figure1C_PLP([1.0, 1.0])

u_train = u
x_train = torch.cat([io.linprog(*f_true(ui), eps=0.0000001).detach().t() for ui in u_train])

fig, ax = f_true.plot(u, color='k', alpha=0.25)
iop.plot_targets(x_train, 'o', color='k', markerfacecolor='white', markersize=10.0, markeredgewidth=1.5, alpha=0.25)
if FIGURE_SAVE_DIR:
    plt.savefig(FIGURE_SAVE_DIR + '/deepinverse_fig1_parametric_1.pdf', bbox_inches='tight', dpi=100)


f_learn = Figure1C_PLP([0.2, 0.4])
fig, ax = f_learn.plot(u, color=[0.1, 0.7, 0.0], alpha=0.25)
iop.plot_targets(torch.cat([io.linprog(*f_learn(ui), eps=0.0000001).detach().t() for ui in u_train]), 'o', markerfacecolor='white', markersize=8.0, markeredgewidth=1.5, color=[0.1, 0.7, 0.0], alpha=0.25)
iop.plot_targets(x_train, 'o', color='k', markerfacecolor='none', markersize=14.0, markeredgewidth=1.5, alpha=0.25)
if FIGURE_SAVE_DIR:
    plt.savefig(FIGURE_SAVE_DIR + '/deepinverse_fig1_parametric_2.pdf', bbox_inches='tight', dpi=100)


lr = [100, 100]
max_steps = 20
io.inverse_parametric_linprog(u_train, x_train, f_learn, lr=lr, max_steps=max_steps,
                              callback=io.inverse_parametric_linprog_step_printer(),
                              solver=io.custom_linprog(eps=0.00001))

fig, ax = f_learn.plot(u, color=[0, 0, 0.9], alpha=0.25)
iop.plot_targets(x_train, 'o', color='k', markerfacecolor='white', markersize=14.0, markeredgewidth=1.5, alpha=0.25)
iop.plot_targets(torch.cat([io.linprog(*f_learn(ui), eps=0.0000001).detach().t() for ui in u_train]), 'o', markerfacecolor='white', markersize=6.0, markeredgewidth=1.5, color=[0.0, 0.0, 0.9], alpha=0.25)
if FIGURE_SAVE_DIR:
    plt.savefig(FIGURE_SAVE_DIR + '/deepinverse_fig1_parametric_3.pdf', bbox_inches='tight', dpi=100)